# การประมวลผลและการเตรียมข้อมูล (Preprocessing)

### รวบรวมข้อมูล

In [ ]:
import json

with open('/content/Fantastic_Beasts_and_Where_to_Find_Them.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
print(f"จำนวน: {len(data)}")

In [ ]:
# ตัวอย่างข้อมูลบางส่วน
print(json.dumps(data[0:5], indent=2, ensure_ascii=False))

### แบ่งส่วนข้อมูล

In [ ]:
# ข้อมูลอยู่ในรูปเเบบเป็นรายละเอียดข้อมูล chunks ย่อย ๆ ของสัตว์มหัศจรรย์เเต่ละตีวอยู่เเล้ว

print("### chunks 1")
print(data[1]["Description"])
print("="*200)
print("### chunks 2")
print(data[2]["Description"])

# Facebook AI Similarity Search (FAISS)

### Setup

In [ ]:
!pip install -qU langchain-community faiss-cpu

### Initialization

In [ ]:
!pip install -qU langchain-huggingface

In [ ]:
!pip install -U FlagEmbedding

In [ ]:
from FlagEmbedding import BGEM3FlagModel
from langchain_huggingface import HuggingFaceEmbeddings

# โหลด embeddings โมเดล
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") # BAAI/bge-m3 รองรับภาษาไทย

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
# ตรวจสอบรายการ UUID ที่ถูกเก็บไว้
print(list(vector_store.index_to_docstore_id.values()))

# [] ยังไม่มีข้อมูล

### Manage vector store

In [ ]:
from uuid import uuid4
from langchain_core.documents import Document

documents = []
for item in data:
  # สร้าง documents store
  document =  Document(
    page_content= item['Description'],
    metadata={
        "Id": item['Id'],
        "Name": item['Name'],
        },
    )
  documents.append(document)

uuids = [str(uuid4()) for _ in range(len(documents))]

# บันทึก UUIDs ลงไฟล์ uuids.json
with open("uuids.json", "w") as f:
    json.dump(uuids, f, indent=2, ensure_ascii=False)

print("บันทึก UUIDs ลงไฟล์ uuids.json เรียบร้อยแล้ว!")


In [ ]:
serialized_documents = [{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in documents]

print(json.dumps(serialized_documents, indent=2, ensure_ascii=False))

In [ ]:
# สร้าง vector store
vector_store.add_documents(documents=documents, ids=uuids)

#### วิธีตรวจสอบว่า UUID อยู่ใน vector_store และดึงข้อมูล

In [ ]:
# ตรวจสอบรายการ UUID ที่ถูกเก็บไว้
uuids = list(vector_store.index_to_docstore_id.values())
print(json.dumps(uuids, indent=1))

In [ ]:
uuid_to_find = "56e8fde8-fa5f-4e77-9747-a4a17803ade6"

# ตรวจสอบว่า UUID อยู่ใน vector_store หรือไม่
if uuid_to_find in vector_store.index_to_docstore_id.values():
    retrieved_doc = vector_store.docstore.search(uuid_to_find)
    print("📄 Document Content:", retrieved_doc.page_content)
    print("📝 Metadata:", retrieved_doc.metadata)
else:
    print("❌ UUID นี้ไม่มีอยู่ใน Vector Store")


#### Delete items from vector store

In [ ]:
print("=== ก่อนลบข้อมูล === ")
uuids = list(vector_store.index_to_docstore_id.values())
print(json.dumps(uuids, indent=1))

print("=== หลังลบข้อมูล ===")
vector_store.delete(ids= [uuids[-1]]) # ลบตัวสุดท้าย
uuids = list(vector_store.index_to_docstore_id.values())
print(json.dumps(uuids, indent=1))

### Query vector store

In [ ]:
results = vector_store.similarity_search_with_score(
    "ฮิปโปกริฟฟ์ มีพื้นเพอยู่ในยุโรป แต่ปัจจุบันพบได้ทั่วโลก มันมีหัวเป็นนกอินทรี ตัวเป็นม้า เลี้ยงให้เชื่องได้ แต่ต้องกระทําโดยผู้เชี่ยวชาญเท่านั้น เมื่อเข้าใกล้ฮิปโปกริฟฟ์จะต้องมองสบตามันไว้ตลอด?", k=10
)
for res, score in results:
    print(f"="*200)
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")
    print(f"* {res.page_content}")
    print(f"* [{res.metadata}]")

In [ ]:
docs = vector_store.similarity_search_with_score(
    "ฉันอยากเสี่ยงสัตววิเศษที่ ที่เป็นมิตร ต่อพ่อมด อาศัยอยู่ในนํ้า ชอบกินผักขม มีตัวอะไรเเนะนำบ้าง", k=20
)

print("docs", docs)

results = []
for res, score in docs:
    results.append(
        {
            "score": f"{score:3f}",
            "content": res.page_content,
            "metadata": res.metadata,
        }

    )

results = {"data": results}
print(json.dumps(results, indent=2, ensure_ascii=False))

with open("result.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [ ]:
results[0]

In [ ]:
# Document
results[0][0]

In [ ]:
# score
results[0][1]

### Saving and loading

In [ ]:
# Saving
vector_store.save_local("faiss_index")

In [ ]:
# สร้าง zip file
!zip -r faiss_index.zip faiss_index/

# ดาวน์โหลดไฟล์ zip
from google.colab import files
files.download('faiss_index.zip')

In [ ]:
# loading
new_vector_store = FAISS.load_local(
    # embeddings ต้องเป็น model เดียวกับกับที่ใช้สร้าง vector_store
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

docs = new_vector_store.similarity_search("qux")

In [ ]:
# ใช้ vector_store ที่เรา save มา
results = new_vector_store.similarity_search_with_score(
    "ฉันสามารถพบฮิปโปกริฟฟ์ได้ที่ไหน", k=2
)
for res, score in results:
    print(f"="*200)
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")
    print(f"* {res.page_content}")
    print(f"* [{res.metadata}]")